# Extracting Z Features from Z type geoshapes (Point/Line/Multiline/Polygon Z Feature Types)
---
Walkthrough on how to extract Z features and append them to the shape file in preperation to burn them as the feature into a raster image.

## Setup Notebook
---

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
import fiona
import geopandas as gpd
import numpy as np

## Import Data
---

In [6]:
plan = '/workspace/data-sets/NYC_DoITT_Planimetric_OpenData.gdb/NYC_DoITT_Planimetric_OpenData.gdb'

In [9]:
# Extract the layer names into a list
layer_list = fiona.listlayers(plan)
layer_list

['PAVEMENT_EDGE',
 'HYDRO_STRUCTURE',
 'RETAININGWALL',
 'HYDROGRAPHY',
 'SIDEWALK',
 'PARK',
 'MEDIAN',
 'SWIMMING_POOL',
 'OPEN_SPACE_NO_PARK',
 'PARKING_LOT',
 'SHORELINE',
 'BOARDWALK',
 'RAILROAD',
 'TRANSPORT_STRUCTURE',
 'ELEVATION',
 'MISC_STRUCTURE_POLY',
 'CURB',
 'ROADBED',
 'PLAZA',
 'SIDEWALK_LINE',
 'RAILROAD_STRUCTURE']

In [10]:
# Extract Layer Data into a List of GPD Frames
plan_data_list = []

for layer_name in layer_list:
    plan_data_list.append(gpd.read_file(plan, layer=layer_name))

## Extract Z From Point
---

In [48]:
print(layer_list[14])
data = plan_data_list[14]
data.head()

ELEVATION


,ELEVATION,SOURCE_ID,FEATURE_CODE,SUB_FEATURE_CODE,STATUS,geometry
0,129.740000,2.130200e+10,3020,302000,Unchanged,POINT Z (989083.000 195003.260 129.740)
1,120.586263,2.130000e+10,3000,300020,Unchanged,POINT Z (987638.301 195007.228 120.586)
2,114.740000,2.130200e+10,3020,302000,Unchanged,POINT Z (988565.410 195011.600 114.740)
3,69.130000,2.130200e+10,3020,302000,Unchanged,POINT Z (989509.570 195013.270 69.130)
4,49.235397,2.130200e+10,3020,302000,Unchanged,POINT Z (987570.796 195019.969 49.235)


In [51]:
z_value = []
for feature in data.geometry:
    # Extract the 3 dimensional features
    z_value.append(tup[2])

In [52]:
print('length of z value enteries:', len(z_value))
print('length of initial gpd:', len(data))

length of z value enteries: 1473788
length of initial gpd: 1473788


In [53]:
data['z-values'] = z_value
data.head()

,ELEVATION,SOURCE_ID,FEATURE_CODE,SUB_FEATURE_CODE,STATUS,geometry,z-values
0,129.740000,2.130200e+10,3020,302000,Unchanged,POINT Z (989083.000 195003.260 129.740),61.5049
1,120.586263,2.130000e+10,3000,300020,Unchanged,POINT Z (987638.301 195007.228 120.586),61.5049
2,114.740000,2.130200e+10,3020,302000,Unchanged,POINT Z (988565.410 195011.600 114.740),61.5049
3,69.130000,2.130200e+10,3020,302000,Unchanged,POINT Z (989509.570 195013.270 69.130),61.5049
4,49.235397,2.130200e+10,3020,302000,Unchanged,POINT Z (987570.796 195019.969 49.235),61.5049


## Extract Z From Multiline
---

In [11]:
print(layer_list[2])
data = plan_data_list[2]
data.head()

RETAININGWALL


,SOURCE_ID,FEATURE_CODE,SUB_FEATURE_CODE,STATUS,SHAPE_Length,height,geometry
0,2.140000e+10,4000,400000,Unchanged,365.715400,41,MULTILINESTRING Z ((985530.573 194711.453 44.4...
1,2.140000e+10,4000,400000,Unchanged,226.155070,66,MULTILINESTRING Z ((985475.003 194408.890 68.2...
2,2.140000e+10,4000,400000,Unchanged,50.949832,19,MULTILINESTRING Z ((985523.914 194769.822 22.7...
3,2.140000e+10,4000,400000,Unchanged,175.276149,65,MULTILINESTRING Z ((987357.515 194593.661 68.9...
4,2.140000e+10,4000,400000,Unchanged,100.051403,69,MULTILINESTRING Z ((987949.170 194143.960 70.9...


Now that the data is loaded we need to  extract the Z coordinates. AS this is is a multiline there are Z values for each pont as part of the multiline. Therefore we need to not only loop over the entries in the gpd but the points which compose the multi line themselves. This will leave us with a list of Z-values for each line.

A quick note - When burning a raster, the Z value can only be one value. We have multiple ways of this and this will be addressed in the raster burning notebook.

In [32]:
z_value = []
for feature in data.geometry:
    # Extract the 3 dimensional features
    coords = [list(line.coords) for line in feature][0]
    z = []
    
    for tup in coords:
        z.append(tup[2])
    z_value.append(z)

### Validation
---

In [33]:
print('length of z value enteries:', len(z_value))
print('length of initial gpd:', len(data))

length of z value enteries: 4078
length of initial gpd: 4078


In [46]:
data['z-values'] = z_value

In [47]:
data.head()

,SOURCE_ID,FEATURE_CODE,SUB_FEATURE_CODE,STATUS,SHAPE_Length,height,geometry,z-values
0,2.140000e+10,4000,400000,Unchanged,365.715400,41,MULTILINESTRING Z ((985530.573 194711.453 44.4...,"[44.40769999999611, 43.77509999999893, 41.4477..."
1,2.140000e+10,4000,400000,Unchanged,226.155070,66,MULTILINESTRING Z ((985475.003 194408.890 68.2...,"[68.25990000000456, 68.26829999999609, 68.9084..."
2,2.140000e+10,4000,400000,Unchanged,50.949832,19,MULTILINESTRING Z ((985523.914 194769.822 22.7...,"[22.720799999995506, 21.874800000005052, 20.94..."
3,2.140000e+10,4000,400000,Unchanged,175.276149,65,MULTILINESTRING Z ((987357.515 194593.661 68.9...,"[68.98029999999562, 68.70500000000175, 64.7836..."
4,2.140000e+10,4000,400000,Unchanged,100.051403,69,MULTILINESTRING Z ((987949.170 194143.960 70.9...,"[70.97000000000116, 70.93670000000566, 70.8925..."
